In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-06-25"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
837,2024-06-25,Porto Rico Bsn,21:00,Mets de Guaynabo,Piratas de Quebradillas,1.82,1.90,173.5,1.81,1.89,0.0,0.00,0.00,t0qrMC1P,0.549451,0.526316,0.552486,0.529101,0.075766,174.600,34.796988,0.199295,3.0,0.000000,0.000000,153.08,1.830,0.200624,0.109631,49.0,166.944,28.707879,0.171961,1.2,1.643168,1.369306,197.73,1.826,0.250260,0.137054,-17.0,86,117,1.78,1.69,147.450,197.486,0.030413,0.030578,NaN,1.63,0.326,2.515337,0.663451,0.8,0.136549,-1.39,-0.278,-3.237410,0.489985,0.3,-0.189985
838,2024-06-25,Mundo Amistosos Internacionais,13:30,Croácia F,Luxemburgo F,1.02,12.50,150.5,1.85,1.85,0.0,0.00,0.00,0lljzmj4,0.980392,0.080000,0.540541,0.540541,0.060392,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,78,0,2.80,0.00,0.000,0.000,1.200826,0.000000,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
839,2024-06-25,Mundo Amistoso Internacional,15:00,República Dominicana,Portugal,1.23,3.86,153.5,1.82,1.88,0.0,0.00,0.00,fiorx9Li,0.813008,0.259067,0.549451,0.531915,0.072075,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0,0,0.00,0.00,0.000,0.000,0.730723,0.022933,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
840,2024-06-25,Colômbia Lbp,19:40,Caribbean Storm Islands,Toros del Valle,1.19,4.30,168.5,1.83,1.83,0.0,0.00,0.00,EZfyzLw8,0.840336,0.232558,0.546448,0.546448,0.072894,127.682,13.134684,0.102870,1.8,1.643168,0.912871,111.32,1.262,0.058907,0.046677,21.0,212.616,128.806002,0.605815,2.4,1.341641,0.559017,440.84,2.230,1.143897,0.512958,13.0,92,107,1.21,4.12,128.130,383.984,0.801130,0.000000,NaN,-2.27,-0.454,-0.418502,0.753605,0.7,-0.053605,-0.88,-0.176,-18.750000,0.000000,0.0,0.000000
841,2024-06-25,Venezuela Superliga,20:30,Gaiteros del Zulia,Centauros,1.27,3.53,149.5,1.86,1.84,0.0,0.00,0.00,GlOaeKkf,0.787402,0.283286,0.537634,0.543478,0.070688,153.460,64.887594,0.422831,1.2,1.643168,1.369306,109.90,2.346,1.299838,0.554066,-3.0,171.666,38.395618,0.223665,0.6,1.341641,2.236068,215.60,2.370,0.558346,0.235589,-34.0,70,70,1.57,3.08,161.908,183.624,0.665859,0.007644,NaN,3.66,0.732,0.368852,0.000000,0.0,0.000000,-1.92,-0.384,-6.588542,0.000000,0.0,0.000000
842,2024-06-25,Paraguai Lnb,20:30,Dep. San Jose,Olimpia,1.66,2.13,152.5,1.85,1.85,0.0,0.00,0.00,YwrpLINr,0.602410,0.469484,0.540541,0.540541,0.071893,117.550,44.004600,0.374348,2.4,1.341641,0.559017,109.34,1.530,0.556956,0.364024,20.0,555.640,1004.347028,1.807550,1.8,1.643168,0.912871,114.55,6.726,11.892852,1.768191,58.0,77,79,1.42,1.45,104.324,568.740,0.175377,0.000000,NaN,1.26,0.252,2.619048,0.000000,0.0,0.000000,27.23,5.446,0.207492,0.000000,0.0,0.000000
843,2024-06-25,Paraguai Lnb,20:30,Colonias Gold,Felix Perez Cardozo,1.23,3.86,148.5,1.82,1.84,0.0,0.00,0.00,KfthJdhe,0.813008,0.259067,0.549451,0.543478,0.072075,201.682,48.816664,0.242048,1.2,1.643168,1.369306,195.84,2.862,0.627272,0.219173,-17.0,682.582,989.091117,1.449044,1.2,1.643168,1.369306,174.87,8.454,11.104273,1.313493,-4.0,72,67,2.72,2.61,229.866,0.000,0.730723,0.007728,NaN,4.54,0.908,0.253304,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
837,21:00,Porto Rico Bsn,Mets de Guaynabo,Piratas de Quebradillas,1.82,Back Home
